In [1]:
import cv2 
import numpy as np 
import imutils
import math

In [2]:

# Let's load a simple image with 3 black squares 
image = cv2.imread('house.png') 
cv2.waitKey(0) 

# Grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

# Find Canny edges 
edged = cv2.Canny(gray, 30, 200) 
cv2.waitKey(0) 

# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged, 
	cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

cv2.imshow('Canny Edges After Contouring', edged) 
cv2.waitKey(0) 

print("Number of Contours found = " + str(len(contours))) 

# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 0, 255), 3) 

cv2.imshow('Contours', image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 


Number of Contours found = 405


In [3]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#print(cnts)
m=0
for c in cnts:
    #print(c)
    #print(len(c))
    #continue
    area=cv2.contourArea(c)
    print(area)
    if area>m:
        #print(c)
        m=area
        M = cv2.moments(c)
        cX = int(M["m10"] / (M["m00"] ))
        cY = int(M["m01"] / (M["m00"]))
    #print(len(c))
    # compute the center of the contour
        #print(cX,cY)

color = np.array(image[cY][cX])
cv2.circle(image, (int(cX), int(cY)), 5, (255, 0, 0), -1)
cv2.imshow('Contours', image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

0.0
147.0
635 589
1.0
0.0
0.0
0.0
490.0
659 581
2.5
1.5
30.5
3.0
0.0
0.0
0.0
951.0
384 558
5.5
1.0
0.0
21.0
6.5
15.5
78.5
3.5
7.5
0.0
436.0
511.5
0.5
0.5
0.5
8.5
0.0
0.0
38.5
3.0
0.0
23151.5
1062 461
0.0
0.0
0.0
37.0
3.5
0.0
2.5
17.0
0.0
3.0
0.0
19089.5
8.0
14.0
13.0
0.0
7.0
2.0
39.0
119.5
14.5
9.0
0.0
0.0
3.0
30.0
6.0
0.0
1.0
0.5
0.5
5.5
0.0
0.0
0.5
10.0
0.0
0.5
13.0
1.5
0.0
0.0
0.0
2.0
3.5
0.0
172.5
0.0
91.5
12.5
0.5
42.0
0.0
464.5
24.5
99.5
24.5
15.0
6.0
178.5
0.0
4.5
3.5
0.5
1.0
30.0
39.5
1292.0
690.0
1.0
7.5
660.5
0.0
669.0
626.5
281.5
21.0
742.0
385.5
2.0
4.0
133.5
40.0
35.5
18.0
2.5
134.5
143.0
3030.0
5.5
1.0
1971.0
8.5
0.0
0.0
0.0
14.0
0.0
19.5
7.0
1.0
17.0
2246.5
11.5
2586.0
3.5
1862.0
6.5
10.5
2.5
342.5
1047.5
0.0
342353.5
514 309


In [4]:
image.shape

(864, 1152, 3)

In [5]:
print(image[125][192]) # Let this distance be 2 m
print(image[110][116]) # Let this distance be 4 m

[47 47 47]
[69 69 69]


In [6]:
color

array([56, 56, 56], dtype=uint8)

In [7]:
# Let 160 -> 2 m, 121 -> 4m
# By Linear Regression, y = x*-0.02820855 + 7.03992702

def colorToDistance(color, slope = -0.02820855, constant = 7.03992702):
    # Simple Linear Regression
    return color*slope + constant

In [8]:
def pixelToComponents(pixel, distance, heading = 0, currentLocation = (0,0,0), image_size = image.shape[:2], max_angleX = math.radians(90)):
    # heading = 0 means heading towards X axis, 90 means Y axis and so on
    # max_x / 2 is equal to heading angle. if this - x = 0, then X,Y = distance*cos(heading), distance*sin(heading)
    # AngleXY 
    max_y, max_x = image_size
    x,y = pixel
    mid_x = max_x / 2
    mid_y = max_y / 2
    deltaAngle = ((x - mid_x) / max_x) * max_angleX
    angleXY = heading - deltaAngle
    angleZ = -((y - mid_y) / max_y) * max_angleX
    return np.multiply(distance, [math.cos(angleXY), math.sin(angleXY), math.sin(angleZ)])

In [9]:
dis = colorToDistance(color)
print(dis)
print(pixelToComponents((cX,cY), dis))

[5.46024822 5.46024822 5.46024822]
[5.44074789 0.46105638 1.2108714 ]


In [10]:
pixelToComponents((200, 194), dis)

array([4.75821278, 2.67838044, 2.28959154])